In [105]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
import os
from pylab import *

# import numpy as np
import os
from matplotlib import gridspec
import scipy.optimize as opt
# import pylab as pl
# from matplotlib import colors, ticker, cm
from scipy import ndimage
# from scipy import optimize 
# from mpl_toolkits.axes_grid1 import make_axes_locatable
from math import pi, e

##Config

In [106]:
AFM_FOLDER = '../data_vault/AFM/'

# For reading .txt files
NLINES = 32
SAMPLES_PER_LINE = 16384

# For plotting color maps
CMIN = -5
CMAX = 5
FIGSIZE = (15,7)

# For determining catalyst heights
SMOOTHNESS = 5 # increase for greater smoothness
MAX_RAD = 30 # neighborhood to search for min

# For histogram
LOW_THRESHOLD = 0.5
HBINS = 50
HMIN = 0
HMAX = 10

##File readers

In [107]:
def read_ascii(file):
    '''
    Reads an ascii file (Nanoscope v5.12). Returns height data as (n x m) array and 
    scan size (width, height) in um.
    '''
    
    f = open(AFM_FOLDER + file, encoding = 'latin-1')
    
    header = True
    hv = 0
    bpp = 0
    imarray = []
    units = ''
    aspect = None
    
    for line in f:
        cols = line.split()
        
        if len(cols) > 1:
            if cols[0] == '\Bytes/pixel:' and bpp == 0: bpp = int(cols[1])
            if cols[0] + cols[1] == '\Scansize:' and units == '': 
                size = float(cols[2])
                units = cols[3]
            if cols[0] + cols[1] == '\@2:Zscale:' and hv == 0: hv = float(cols[-2])
            if cols[0] == '\Samps/line:': samps_per_line = int(cols[1])
            if cols[0] == '\Lines:': lines = int(cols[1])
            if cols[0] == '\Aspect' and aspect is None: aspect = int(cols[2].split(':')[0])
        
        if not header and len(cols) > 1:
            scale = hv/(2**(8*bpp))**2/2*25e5 # in nm
            #image stored as rows, columns (upright)
            imarray.append([float(i)*scale for i in cols]) 
            
        if line.strip() == "": 
            header = False
    
    f.close()
    
    # scan size (width, height) in um
    size = (size / 1000, size / aspect / 1000)
    
    # cuts out amplitude data
    imarray = array(imarray[0:len(imarray)//2])
    
    # if rectangular, cuts out margins at top and bottom 
    if aspect != 1:
        n, m = shape(imarray)
        imarray = imarray[int(m*(0.5-1/(2*aspect))):int(m*(0.5+1/(2*aspect))), :]
    
    # flips image vertically
    imarray = flipud(imarray)
    
    return imarray, size

In [108]:
def read_txt(filename, nlines = NLINES, samples_per_line = SAMPLES_PER_LINE):
    '''
    Reads a simple text file from CNF AFM. Returns height data as 
    (nlines x samples_per_line) array.
    '''
    
    #Loads data
    data = np.loadtxt(AFM_FOLDER + filename,skiprows=1)

    #Load in catalyst heights then reshape into a matrix
    CHeights = np.copy(data[:,0])
    CHeights = CHeights.reshape(nlines,samples_per_line)
    
    CHeights = flipud(CHeights)
    
    return CHeights

##Color map

In [109]:
def plot_image(im, size = None, filename = None, save = False,
               cmin = CMIN, cmax = CMAX, ax = None):
    '''
    Plots the height data as a color map. 
    
    im: (m x n) array of height data
    size: (width, height) in um; if None, size is the dimensions of im in pixels
    '''
    
    if size is None: size = (shape(im)[1], shape(im)[0])
    
    if ax is None:
        fig = figure(figsize = FIGSIZE)
        ax = fig.gca()
    
    img = ax.imshow(im, aspect = 'equal', interpolation = 'none',
                extent = (0,size[0],size[1],0), 
                cmap = get_cmap('afmhot'))

    if size != shape(im)[::-1]: ax.set_xlabel('Position (um)')
        
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = colorbar(img, cax=cax)
    img.set_clim(cmin, cmax)
    ylabel('Height (nm)', fontsize = 14)   
    
    if save and filename is not None:
        savefig(AFM_FOLDER + str(filename) + '.png',
            bbox_inches = 'tight', pad_inches= 0.2)
        
    return ax

In [110]:
def ascii_to_image(file, save = False):
    imarray, pixel_size, units = read_file(file)
    plot_image(imarray, pixel_size, units, file, save)

In [111]:
def convert_all_ascii():
    for file in os.listdir(AFM_FOLDER):
        if '.' in file and file[0] != '.' and not (file + '.png') in os.listdir(AFM_FOLDER):
            print(file)
            ascii_to_image(file, save = True)
    close()

#Catalyst Size Analysis

##Local maximum finder

In [112]:
# Source: http://stackoverflow.com/questions/3684484/peak-detection-in-a-2d-array/3689710#3689710
from scipy.ndimage.filters import maximum_filter
from scipy.ndimage.morphology import generate_binary_structure, binary_erosion

def detect_peaks(image):
    """
    Takes an image and detect the peaks using the local maximum filter.
    Returns a boolean mask of the peaks (i.e. 1 when
    the pixel's value is the neighborhood maximum, 0 otherwise)
    """

    # define an 8-connected neighborhood
    neighborhood = generate_binary_structure(2,2)

    #apply the local maximum filter; all pixel of maximal value 
    #in their neighborhood are set to 1
    local_max = maximum_filter(image, footprint=neighborhood)==image
    #local_max is a mask that contains the peaks we are 
    #looking for, but also the background.
    #In order to isolate the peaks we must remove the background from the mask.

    #we create the mask of the background
    background = (image==0)

    #a little technicality: we must erode the background in order to 
    #successfully subtract it form local_max, otherwise a line will 
    #appear along the background border (artifact of the local maximum filter)
    eroded_background = binary_erosion(background, structure=neighborhood, border_value=1)

    #we obtain the final mask, containing only peaks, 
    #by removing the background from the local_max mask
    detected_peaks = local_max - eroded_background

    return detected_peaks

##Determine Centers of Catalysts¶

In [113]:
def get_circle(center, rad, im = None, size = None, npts = 50):
    
    if im is not None and size is None: size = shape(im)[::-1]
        
    m, n = shape(im)
    
    aspect = (n/size[0])/(m/size[1])
    
    xs = center[1] + (rad*cos(linspace(0,2*pi,npts))).astype(int)
    ys = center[0] + (rad/aspect*sin(linspace(0,2*pi,npts))).astype(int)
    
    circle = zip(ys, xs)
    circle = filter(lambda loc: min(loc) >= 0 and loc[0] < m and loc[1] < n, circle)
    
    return circle

In [114]:
def locate_cats(im, smoothness = SMOOTHNESS, max_rad = MAX_RAD, size = None, ax = None):
    '''
    Finds the location and heights catalysts in an AFM image. 
    Returns a list of dictionaries.
    
    im: the starting image (without smoothing)
    smoothness: controls the amount of smoothing; lower for less smoothing
    '''
    
    m,n = shape(im)
    
    # Smooths out image for peak finding algorithm only in direction
    # Lower sigma for less smoothing and increase for more averaging
    smooth = ndimage.filters.gaussian_filter(im, sigma = [n/m*smoothness, smoothness])

    # Uses peak detection algorithm
    result = detect_peaks(smooth)
    ys,xs = where(result==1)
    cats = [{'loc':loc} for loc in zip(list(ys),list(xs))]
    
    for cat in cats:
        maxH = im[cat['loc']]
        minH = maxH
        minRad = 0
        
        for rad in range(1, max_rad):
            circle = get_circle(cat['loc'], rad, im = im, size = size)
            
            h = mean([im[loc] for loc in circle])
            
            if h < minH:
                minRad = rad
                minH = h
                
        cat['height'] = maxH - minH
        cat['rad'] = minRad
    
    return cats

##Plot results

In [115]:
def plot_cat_map(im, cats, size = None, ax = None):
    m, n = shape(im)
    
    if size is None: size = (n, m)
    
    ax = plot_image(im, size = size, ax = ax)
    
    for cat in cats:
        ax.plot(cat['loc'][1]*size[0]/n, cat['loc'][0]*size[1]/m, 'b.', markersize = 2)
        
        if 'rad' in cat:
            circle = get_circle(cat['loc'], cat['rad'], im = im, size = size)
            
            # converts coordinates from pixels to image scale (e.g. um)
            circle = [(loc[1]*size[0]/n, loc[0]*size[1]/m) for loc in circle]
            
            ax.plot(*zip(*circle), color = 'b', markersize = 0.5, alpha = 0.5)
        
    ax.margins(0,0)

In [116]:
def plot_height_dist(cats, hmin = HMIN, hmax = HMAX, hbins = HBINS, ax = None):
    if ax is None:
        fig = figure() 
        ax = fig.gca()
    
    heights = [cat['height'] for cat in cats]
    
    ax.hist(heights, bins = hbins,range=(hmin, hmax));
    ax.set_xlabel('Height (nm)')
    ax.set_ylabel('Counts')

In [117]:
def plot_all(im, size = None, smoothness = SMOOTHNESS, max_rad = MAX_RAD, figsize = FIGSIZE,
              hmin = HMIN, hmax = HMAX, hbins = HBINS, save = False, fn = None):
    
    figure(figsize = figsize)
    gs = gridspec.GridSpec(1, 2, width_ratios = [1.2, 1], wspace = 0.25)
    ax1 = subplot(gs[0,0])
    ax2 = subplot(gs[0,1])

    cats = locate_cats(im, smoothness = smoothness, max_rad = max_rad)
    plot_cat_map(im, cats, size = size, ax = ax1)
    plot_height_dist(cats, hmin = hmin, hmax = hmax, hbins = hbins, ax = ax2)
    
    if save and filename is not None:
        savefig(AFM_FOLDER + str(filename) + '_cats.png',
            bbox_inches = 'tight', pad_inches= 0.2)
        
    return cats